In [66]:
import pandas as pd
import numpy as np

# Transforming Data

The bread and butter of a data workflow is cleaning and preparation, taking raw datasets and transforming them into a useful form.

Today, we'll use some prescription cost data from the NHS to make a chart of the top 50 most used Cardiovascular substances.

</br></br></br></br>


## Introducing Tools: Pandas

The first tool we'll use today is `Pandas`, a Python library used to work with datasets. It provides access to `DataFrames` - tables we analyse with code.

Python already has a few built in data structures, for example lists and dictionaries:

</br></br></br>

In [58]:
london = {
    "name": "London",
    "population": 8308369,
    "area": 1572
} # This is an example of a dictionary

locations = [
    {
        "name": "London",
        "population": 8_982_000,
        "area": 606
    },
    {
        "name": "Newport",
        "population": 128_060,
        "area": 32.52
    },
    {
        "name": "Darlington",
        "population": 93_015,
        "area": 7.62
    },

]


<br>
<br>
<br>
<br>
Which we can turn into Pandas `DataFrames`

In [59]:
df = pd.DataFrame(locations)
df

,name,population,area
0,London,8982000,606.00
1,Newport,128060,32.52
2,Darlington,93015,7.62


<br>
<br>
<br></br></br>

<br>

and manipulate in different ways.

For example, we can add a density column:

In [60]:
df['density'] = df['population'] / df['area']
df

,name,population,area,density
0,London,8982000,606.00,14821.782178
1,Newport,128060,32.52,3937.884379
2,Darlington,93015,7.62,12206.692913


In [61]:
sorted_df = df.sort_values(by="density", ascending=False)
sorted_df

,name,population,area,density
0,London,8982000,606.00,14821.782178
2,Darlington,93015,7.62,12206.692913
1,Newport,128060,32.52,3937.884379


# Participant Data: CHAPS Volume

Let's start by loading the data and setting column names.

In [62]:
df = pd.read_excel("sample_data/historic-monthly-payments-statistics.xls", skiprows=7, usecols=range(1,11))
df.columns = [
    "year", "month", "gbp_volume", "euro_volume_domestic", "euro_volume_target", "empty",  "empty",  "gbp_value", "euro_value_domestic", "euro_value_target"
]



</br></br></br>

We only care about the date and the value - let's filter for these and format the date.

In [63]:
df = df[['year', 'month', 'gbp_value']] # we only need the year, month and gbp_value columns

# drop rows without values
df = df.dropna(subset=['gbp_value'])

# fill forward the year
df['year'] = df['year'].ffill()

# make a date column, merging the year and month columns
df['date'] = pd.to_datetime(df['year'].astype(str) + '-' + df['month'].astype(str) + '-01')

# keep only the date and gbp_value columns
df = df[['date', 'gbp_value']]

# multiply the gbp_value by 1,000 - it's in £000s at the moment
df['gbp_value'] = df['gbp_value'] * 1000


</br></br></br>

We also want the change - we can use panda's `pct_change` function.

In [64]:

# calculate the year-on-year change
df['change'] = df['gbp_value'].pct_change(12)

df = df.melt(id_vars=['date'], value_vars=['gbp_value', 'change'], var_name='series', value_name='value')


</br></br></br>

Finally, we can export the data.

In [65]:

df.to_csv("sample_data/chaps_values.csv", index=False)